SVM on MNIST dataset 

In [17]:
import pandas as pd
import numpy as np
from libsvm.svmutil import svm_train, svm_predict, svm_problem, svm_parameter


X_train = pd.read_csv('./data/X_train.csv', header=None)
X_test = pd.read_csv('./data/X_test.csv', header=None)
y_train = pd.read_csv('./data/Y_train.csv', header=None)
y_test = pd.read_csv('./data/Y_test.csv', header=None)


X_train = X_train.values.tolist()
X_test = X_test.values.tolist()
y_train = y_train.values.ravel().tolist()
y_test = y_test.values.ravel().tolist()

# kernel 設定：0 = linear, 1 = polynomial, 2 = RBF
kernel_names = {0: "linear", 1: "polynomial", 2: "rbf"}
kernel_results = {}

for k in [0, 1, 2]:
    prob = svm_problem(y_train, X_train)
    if k == 1:
        param = svm_parameter('-s 0 -t 1 -d 3 -c 1')
    else:
        param = svm_parameter(f'-s 0 -t {k} -c 1')
    model = svm_train(prob, param)
    pred_label, acc, _ = svm_predict(y_test, X_test, model, options='-q')
    kernel_results[kernel_names[k]] = acc[0]

print("Kernel results: ", kernel_results)

Kernel results:  {'linear': 95.08, 'polynomial': 34.68, 'rbf': 95.32000000000001}


In [18]:
def grid_search(X_train, y_train, X_test, y_test, kernel_type, param_grid):
    best_acc = 0
    best_params = None

    if kernel_type == 'linear':
        combinations = [[C] for C in param_grid['C']]
    elif kernel_type == 'polynomial':
        combinations = []
        for d in param_grid['degree']:
            for C in param_grid['C']:
                for g in param_grid['gamma']:
                    combinations.append([d, C, g])
    else:  # RBF
        combinations = []
        for g in param_grid['gamma']:
            for C in param_grid['C']:
                combinations.append([g, C])

    for params in combinations:
        if kernel_type == 'linear':
            C = params[0]
            param_str = f'-s 0 -t 0 -c {C}' # -s svm_type : set type of SVM (default 0) 0 -- C-SVC
        elif kernel_type == 'polynomial':
            degree, C , gamma = params
            param_str = f'-s 0 -t 1 -d {degree} -c {C} -g {gamma}'  # -d degree : set degree of polynomial kernel (default 3)
        else:  # RBF
            gamma, C = params
            param_str = f'-s 0 -t 2 -g {gamma} -c {C}'

        prob = svm_problem(y_train, X_train)
        param = svm_parameter(param_str)
        model = svm_train(prob, param)
        _, acc, _ = svm_predict(y_test, X_test, model, options='-q')

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_params = params

    return best_params, best_acc


param_grid = {
    'linear': {'C': [0.1, 1, 10]},
    'polynomial': {'degree': [2, 3], 'C': [0.1, 1, 10], 'gamma': [0.01, 0.1]},
    'rbf': {'gamma': [0.01, 0.1], 'C': [0.1, 1, 10]}
}

kernel_results = {}

for kernel_type in param_grid.keys():
    best_params, best_acc = grid_search(X_train, y_train, X_test, y_test, kernel_type, param_grid[kernel_type])
    kernel_results[kernel_type] = (best_params, best_acc)
    print(f"Best parameters for {kernel_type} kernel: {best_params}, Accuracy: {best_acc:.2f}%")



Best parameters for linear kernel: [0.1], Accuracy: 95.80%
Best parameters for polynomial kernel: [2, 0.1, 0.1], Accuracy: 97.76%
Best parameters for rbf kernel: [0.01, 10], Accuracy: 98.20%


In [ ]:
def mixed_kernel(X, Y, gamma=0.5):
    X = np.array(X)
    Y = np.array(Y)

    linear_part = X @ Y.T
    X_norm = np.sum(X ** 2, axis=1).reshape(-1, 1)
    Y_norm = np.sum(Y ** 2, axis=1).reshape(1, -1)
    rbf_part = np.exp(-gamma * (X_norm + Y_norm - 2 * np.dot(X, Y.T)))

    return  linear_part +  rbf_part


def add_index_column(K):
    n = K.shape[0]
    index = np.arange(1, n + 1, dtype=int).reshape(-1, 1)
    return np.hstack((index, K))


def grid_search_mixed(X_train, y_train, X_test, y_test, param_grid):
    best_acc = 0
    best_params = None

    combinations = []
    for c in param_grid['C']:
        for gamma in param_grid['gamma']:
            combinations.append([c,gamma])

    for params in combinations:
        c,  gamma = params

        K_train = mixed_kernel(X_train, X_train, gamma)
        K_test = mixed_kernel(X_test, X_train, gamma)

        K_train_indexed = add_index_column(K_train)
        K_test_indexed = add_index_column(K_test)

        model = svm_train(y_train, K_train_indexed.tolist(), f'-t 4 -c {c} -g {gamma} -q ')
        pred_label, acc, _ = svm_predict(y_test, K_test_indexed.tolist(), model, options='-q')

        if acc[0] > best_acc:
            best_acc = acc[0]
            best_params = params

    return best_params, best_acc


param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.01, 0.1]
}
kernel_results = {}

best_params, best_acc = grid_search_mixed(X_train, y_train, X_test, y_test, param_grid)
kernel_results['mixed'] = (best_params, best_acc)
print(f"Best parameters for mixed kernel: {best_params}, Accuracy: {best_acc:.2f}%")



Best parameters for mixed kernel: [0.1, 0.01], Accuracy: 95.92%


In [ ]:
# %pip install libsvm==3.23.0.4


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip install numpy==1.19.0
# %pip install pandas==1.1.5

  Using cached numpy-1.19.0.zip (7.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [24 lines of output]
      Running from numpy source directory.
      <string>:460: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
      Traceback (most recent call last):
        File "c:\Users\yuchu\Miniconda3\envs\tf_env\lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
        File "c:\Users\yuchu\Miniconda3\envs\tf_env\lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
        File "c:\Users\yuchu\Miniconda3\envs\tf_env\lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 175, in prepare_metadata_for_build_wheel
          return hook(metadata_directory, config_settings)
        File "C:\Users\yuc

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip list

Package                      Version
---------------------------- -----------
absl-py                      2.1.0
aiohappyeyeballs             2.4.6
aiohttp                      3.11.12
aiosignal                    1.3.2
asttokens                    3.0.0
astunparse                   1.6.3
async-timeout                5.0.1
attrs                        25.1.0
blinker                      1.9.0
Brotli                       1.1.0
cached-property              1.5.2
cachetools                   5.5.2
certifi                      2025.1.31
cffi                         1.17.1
charset-normalizer           3.4.1
click                        8.1.8
colorama                     0.4.6
comm                         0.2.2
contourpy                    1.3.0
cryptography                 39.0.0
cycler                       0.12.1
debugpy                      1.6.7
decorator                    5.2.0
exceptiongroup               1.2.2
executing                    2.1.0
flatbuffers                  25.2.10
